In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_miocardial()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.6)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (1700, 110)
5.62% missing data
Class distribution: (array([0, 1]), array([1429,  271]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 2),
                       (46, 9, 1),
                       (96, 9, 5),
                       (96, 11, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 2.38 sec - Loss 0.133102 - ACC 66.58% - ACC Mean 66.58% - AUC 72.94% - AUC Mean 72.94% - Deter 000
Ite 00050 - 0.88 sec - Loss 0.044479 - ACC 66.46% - ACC Mean 66.51% - AUC 69.34% - AUC Mean 70.67% - Deter 046
Ite 00054 - 0.98 sec - Loss 0.041818 - ACC 66.60% - ACC Mean 66.51% - AUC 71.20% - AUC Mean 70.56% - Deter 050
Early stop ite 54, rollback to correction of ite 4, whith acc of 67.83% and auc of 73.38%
OUR METHOD RUN 1/10 - acc: 67.83% +- 0.0% - auc: 73.38% +- 0.0%
Ite 00000 - 1.15 sec - Loss 0.151212 - ACC 66.70% - ACC Mean 66.70% - AUC 71.81% - AUC Mean 71.81% - Deter 000
Ite 00050 - 1.07 sec - Loss 0.065200 - ACC 66.18% - ACC Mean 67.87% - AUC 70.81% - AUC Mean 72.32% - Deter 015
Ite 00085 - 0.89 sec - Loss 0.040022 - ACC 67.32% - ACC Mean 67.63% - AUC 70.66% - AUC Mean 72.25% - Deter 050
Early stop ite 85, rollback to correction of ite 35, whith acc of 68.14% and auc of 75.12%
OUR METHOD RUN 2/10 - acc: 68.14% +- 0.0% - auc: 75.12% +- 0.0%
Ite 00000 - 1.08 sec - Lo

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 70.39% +- 0.0% - auc: 76.05% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 50.0% +- 0.0% - auc: 52.31% +- 1.85%
SFIL RUN 2/10 - acc: 50.0% +- 0.0% - auc: 52.88% +- 2.86%
SFIL RUN 3/10 - acc: 50.0% +- 0.0% - auc: 53.54% +- 1.6%
SFIL RUN 4/10 - acc: 50.15% +- 0.29% - auc: 55.43% +- 2.47%
SFIL RUN 5/10 - acc: 50.0% +- 0.0% - auc: 53.72% +- 1.85%
SFIL RUN 6/10 - acc: 52.3% +- 4.61% - auc: 57.61% +- 5.03%
SFIL RUN 7/10 - acc: 50.0% +- 0.0% - auc: 53.9% +- 1.96%
SFIL RUN 8/10 - acc: 50.74% +- 1.48% - auc: 53.82% +- 2.24%
SFIL RUN 9/10 - acc: 50.0% +- 0.0% - auc: 52.82% +- 2.43%
SFIL RUN 10/10 - acc: 52.8% +- 5.61% - auc: 54.92% +- 6.0%
SFIL GLOBAL - acc: 50.6% +- 1.01% - auc: 54.1% +- 1.47%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 71.48% +- 3.25% - auc: 74.1% +- 2.55%
PFIL RUN 2/10 - acc: 71.49% +- 4.02% - auc: 74.98% +- 4.32%
PFIL RUN 3/10 - acc: 70.23% +- 3.18% - auc: 73.24% +- 4.49%
PFIL RUN 4/10 - acc: 71.1% +- 3.14% - auc: 75.94% +- 3.12%
PFIL RUN 5/10 - acc: 71.15% +- 1.32% - auc: 74.58% +- 1.56%
PFIL RUN 6/10 - acc: 71.47% +- 1.85% - auc: 76.05% +- 4.1%
PFIL RUN 7/10 - acc: 70.89% +- 0.9% - auc: 74.08% +- 2.68%
PFIL RUN 8/10 - acc: 70.87% +- 3.59% - auc: 73.32% +- 3.3%
PFIL RUN 9/10 - acc: 70.34% +- 3.28% - auc: 72.39% +- 1.92%
PFIL RUN 10/10 - acc: 71.51% +- 4.24% - auc: 74.61% +- 4.89%
PFIL GLOBAL - acc: 71.05% +- 0.45% - auc: 74.33% +- 1.1%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 57.93% +- 7.32% - auc: 61.55% +- 6.08%
SPOL RUN 2/10 - acc: 53.42% +- 2.86% - auc: 54.99% +- 3.69%
SPOL RUN 3/10 - acc: 51.85% +- 3.39% - auc: 55.45% +- 8.09%
SPOL RUN 4/10 - acc: 54.25% +- 4.9% - auc: 58.21% +- 8.71%
SPOL RUN 5/10 - acc: 55.24% +- 3.17% - auc: 61.0% +- 7.82%
SPOL RUN 6/10 - acc: 51.97% +- 1.71% - auc: 54.35% +- 2.59%
SPOL RUN 7/10 - acc: 53.25% +- 2.6% - auc: 56.42% +- 2.69%
SPOL RUN 8/10 - acc: 56.84% +- 4.88% - auc: 59.99% +- 6.66%
SPOL RUN 9/10 - acc: 54.75% +- 3.09% - auc: 58.04% +- 6.41%
SPOL RUN 10/10 - acc: 55.33% +- 3.52% - auc: 65.01% +- 5.18%
SPOL GLOBAL - acc: 54.48% +- 1.87% - auc: 58.5% +- 3.22%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 67.87% +- 5.63% - auc: 73.69% +- 5.69%
PPOL RUN 2/10 - acc: 71.1% +- 2.66% - auc: 72.17% +- 2.53%
PPOL RUN 3/10 - acc: 70.14% +- 2.66% - auc: 74.85% +- 2.96%
PPOL RUN 4/10 - acc: 69.47% +- 4.85% - auc: 73.23% +- 6.74%
PPOL RUN 5/10 - acc: 70.38% +- 0.78% - auc: 72.92% +- 1.84%
PPOL RUN 6/10 - acc: 69.93% +- 2.7% - auc: 72.57% +- 2.06%
PPOL RUN 7/10 - acc: 66.89% +- 4.14% - auc: 71.64% +- 3.01%
PPOL RUN 8/10 - acc: 72.38% +- 4.12% - auc: 75.23% +- 4.19%
PPOL RUN 9/10 - acc: 68.76% +- 2.26% - auc: 72.88% +- 1.88%
PPOL RUN 10/10 - acc: 69.3% +- 2.44% - auc: 72.08% +- 2.74%
PPOL GLOBAL - acc: 69.62% +- 1.49% - auc: 73.13% +- 1.11%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly worse than MICE
Metric AUC - OUR METHOD is significantly worse than MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is significantly worse than PFIL
Metric AUC - OUR METHOD is even with PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is significantly worse than PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
